In [ ]:
# Need this for this demo to even be possible,
# turns out that jupyter already runs in its own
#  event loop which is NOT pretty to deal with
import nest_asyncio
nest_asyncio.apply()
import asyncio

%load_ext memory_profiler

In [ ]:
import requests

print(requests.get("http://0.0.0.0:8080").text)
input()
print(requests.get("http://0.0.0.0:8080/items").text[:1000])
input()
print(requests.get("http://0.0.0.0:8080/items/1").text)

In [ ]:
for _ in range(6500):
    print(requests.get("http://0.0.0.0:8080/items/1").text)

In [ ]:
from aiohttp import request
import asyncio

async def req(i):
    async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
        print(await r.content.read())

asyncio.run(req(1))

In [ ]:
from throttler import Throttler

async def req(i, t: Throttler):
    print(i, "starting")
    async with t:
        print(i, "inside throttler")
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            print(i, "before request")
            resp = await r.content.read()
            print(i, "after request")
            return resp

t = Throttler(rate_limit=60, period=10.0)
results = asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(70)]))

In [ ]:
from tenacity import retry, stop_after_attempt, before_log

from throttler import Throttler

import logging

logging.basicConfig(stream=sys.stderr, level=logging.DEBUG)

logger = logging.getLogger(__name__)

# @retry(stop=stop_after_attempt(3), before=before_log(logger, logging.DEBUG))
@retry(stop=stop_after_attempt(3))
async def req(i, t: Throttler):
    async with t:
        async with request("get", f"http://0.0.0.0:8080/items/{i}") as r:
            resp = await r.content.read()
            r.raise_for_status()
            return resp

def run_reqs(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    asyncio.run(asyncio.gather(*[asyncio.create_task(req(i, t)) for i in range(n_reqs)]))

def run_reqs_2(rate_limit: int, period: float, n_reqs: int):
    t = Throttler(rate_limit=rate_limit, period=period)
    tasks = [req(i, t) for i in range(n_reqs)]

async def testr(n):
    return n

def create_tasks(n):
    tasks = [asyncio.create_task(testr(i)) for i in range(n)]

def create_tasks_2(n):
    tasks = [testr(i) for i in range(n)]

In [ ]:
# %memit run_reqs(10, 1.0, 1000)
%memit run_reqs_2(10, 1.0, 1000)
# %memit run_reqs_3(10, 1.0, 1000)
%memit create_tasks(1000)
%memit create_tasks(5000)
%memit create_tasks(20_000)
%memit run_reqs_2(10, 1.0, 20_000)

In [ ]:
import asyncio
import time

class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, debug=False, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.lock = asyncio.Lock()
        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        self.debug = debug
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def notify(self):
        """
        Signals to the queue that an item is being retried, 
        so pause any get()s by aquiring the lock and throttling before releasing
        """
        async with self.lock:
            await self._throttle()

    async def lock(self, n_seconds: int):
        async with self.lock:
            await asyncio.sleep(n_seconds)

    async def get(self):
        async with self.lock:
            await self._throttle()
            result = await super(ThrottledQueue, self).get()

            self.last_get = time.time()
            return result

    async def retry(self):
        async with self.lock:
            await self._throttle()

    async def _throttle(self):
        elapsed = time.time() - self.last_get
        sleep_time = (1/float(self.per_second)) - elapsed
        if self.debug:
            print(self.i, '- times', f'{elapsed:.5f}', '+', f'{sleep_time:.5f}', '=', self.per_second, '- sizes', self.qsize(), f'{self.qsize() / max(1, self.maxsize):.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds

In [1]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    retrying = False
    while True:
        if not retrying:
            i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            # TODO: actually retry lol, don't just pop a fresh item
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def gen_req(idx): pass

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [2]:
run(100, n_workers=10)

NameError: name 'ThrottledQueue' is not defined

In [ ]:
with open("results.txt", "w") as ostream:
    run(per_second=100, n_workers=100, ostream=ostream, debug=True)

In [ ]:
run(per_second=100, n_workers=10, debug=True)

## Fill the Queue beforehand

In [ ]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    while True:
        i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr

In [ ]:
import asyncio
async def fill_queue(q, items):
    for idx, i in enumerate(items):
        await q.put((idx, i))
    await q.put((idx, Sentinel))
    return q

In [ ]:
from itertools import count

async def _unpack_queue(q):
    l = list()
    for idx in count():
        i, d = await q.get()
        if d == Sentinel:
            print("exiting")
            return l
        l.append(d)
    return l

def unpack_queue(q):
    return asyncio.run(asyncio.create_task(_unpack_queue(q)))


In [ ]:
q = asyncio.run(fill_queue(asyncio.Queue(), list(range(1000))))

print(q.qsize(), sys.getsizeof(q))

l = unpack_queue(q)

print(q.qsize(), sys.getsizeof(q), sys.getsizeof(l), q.qsize(), len(l))

## A more complete example

- Logging before/after request
- Stats collection
  - Request duration
  - Number of retries
  - Errors received
- Rate limited/throttled requests
  - Ability to throttle ALL coroutines on demand (e.g. 503)
- Custom error handlers
- Custom request builders (build endpoint URL/request data from something like an endpoint ID)
- Be able to join multiple consumers/producers together with Queue in between
  - Or just have a single consumer working on a single queue and printing results

In [1]:
import json

from collections import Counter

import sys
from aiohttp import ClientResponseError

class Sentinel: pass

async def get_all_items(q, cntr: Counter):
    async with request("get", "http://0.0.0.0:8080/items") as r:
        resp = await r.read()
        r.raise_for_status()
        cntr["success"] += 1
        for i, d in enumerate(json.loads(resp)):
            await q.put((i, d))
        await q.put((i, Sentinel))


async def handle_error(e, q):
    print(f"HANDLING ERROR: {e}")
    if e.status == 429:
        q.retry()
    elif e.status == 503:
        q.lock(10)

async def item_worker(q, idx, cntr: Counter, ostream = sys.stdout):
    retrying = False
    while True:
        if not retrying:
            i, d = await q.get()
        if d == Sentinel:
            await q.put((i, Sentinel))
            print(f"worker {idx} exiting")
            return
        try:
            # TODO: actually retry lol, don't just pop a fresh item
            async with request("get", f"http://0.0.0.0:8080/items/{d}") as req:
                resp = await req.read()
                req.raise_for_status()
                print(f"worker {idx} response for #{i}: {resp}", file=ostream)
                cntr["success"] += 1
        except ClientResponseError as e:
            await handle_error(e, q)

def gen_req(idx): pass

def run(per_second, n_workers=10, ostream=sys.stdout, debug=False):
    cntr = Counter()
    q = ThrottledQueue(per_second=per_second, maxsize=1000, debug=debug)
    asyncio.run(
        asyncio.gather(
            asyncio.create_task(get_all_items(q, cntr)),
            *[asyncio.create_task(item_worker(q, i, cntr, ostream)) for i in range(n_workers)],
        )
    )
    return cntr